# Gaussian Process Regression in Theory

## Necessary setup for Google Colab & imports

In [ ]:
! git clone https://github.com/ml-kiwi-com/ml-prague.git
! pip install -r ml-prague/requirements.txt

In [ ]:
%cd ml-prague

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel
from sklearn.model_selection import train_test_split
from src import plots
from src import utils

## Prior Distribution

In [ ]:
x = np.linspace(0, 5, 100)
n_samples = 5
utils.generate_prior_gps(x, n_samples, std=9, length_scale=2)

## Posterior Distribution

In [ ]:
x_train = np.linspace(0, 5, 10)
x_grid = np.linspace(0, 5, 100)
y_train_actual = np.sin((x_train - 2.5) ** 2)
n_samples = 5

In [ ]:
utils.generate_posterior_gps(x_train,x_grid, y_train_actual, n_samples)

## Posterior Distribution & Predictor

In [ ]:
utils.generate_posterior_gps(x_train,x_grid, y_train_actual, n_samples, show_predictor=True)

## Toy Function

Let's use a nice 1D function as an example:

$$ f(x) = (x \cdot (10 - x)) \cdot sin(2x) $$

In [ ]:
xlim = [0, 10]
x_nbr = 1_000
X = np.linspace(start=xlim[0], stop=xlim[1], num=x_nbr).reshape(-1, 1)
y = utils.obj_function(X)

Pick training data - small number of samples as we assume they come a from heavy simulator.

In [ ]:
train_size = 7
X_train, _X_test, y_train, _y_test = train_test_split(
    X, y, train_size=train_size, random_state=3
)

## Implementation in *sklearn*

We will use the [sklearn.gaussian_process.GaussianProcessRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessRegressor.html#sklearn.gaussian_process.GaussianProcessRegressor) class.

You can also use the scikit-learn [user guide](https://scikit-learn.org/stable/modules/gaussian_process.html#gaussian-process) for future reference.

## Kernel Introduction

There are multiple kernels available in the sklearn implementation, for now, we will use the following:
- ConstantKernel
- RBF (Radial basis function)

Kernel interaction is already natively implemented in the package, so we can use notation like:
- `kernel_1 * kernel_2` for kernel multiplication
- `kernel_1 + kernel_2` for kernel addition
- etc

## Fitting GPR

Now let's fit the GPR using *RBF* kernel
- define kernel
- initialize the GPR
- fit kernel hyperparameters: variance & length of scale

In [ ]:
kernel = 1 * RBF(length_scale=1.0, length_scale_bounds=(.1, 1e2))
gaussian_process = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)
gaussian_process.fit(X_train, y_train)
gaussian_process.kernel_

Using the fitted kernel, we can generate GPR prediction.

In [ ]:
kriging_mean, kriging_std = gaussian_process.predict(X, return_std=True)
plots.show_basic_kriging_plot_1D(
    X,
    y,
    X_train,
    y_train,
    kriging_mean,
    kriging_std,
    "Gaussian process regression on noise-free dataset",
)


What does our GPR look like?

In [ ]:
gaussian_process.kernel_.get_params()

## Model Quality Metrics

### Metrics for Kriging mean

Standard regression metrics can be utilized for Kriging mean.
- MSE / RMSE
- MAE
- R-squared

The choice of the metric will always up to you as it should reflect the use case for which you are building the model.

For the rest of this workshop, we will be using RMSE as our main metric for the Kriging mean.

In [ ]:
rmse_basic = round(utils.rmse(y, kriging_std, xlim, x_nbr), 2)
mae_basic = round(utils.mae(y, kriging_std, xlim, x_nbr), 2)
print(f"Observed metrics \n \t RMSE: {rmse_basic} \n \t MAE: {mae_basic}")

## Sequential Design

What should be the next point that we should generate so that we improve our model most?

It depends what *most* means for us.

### Local Approach

Improve the model in the area where it performs worst.

First, let's recall where we're at.

In [ ]:
plots.show_basic_kriging_plot_1D(
    X,
    y,
    X_train,
    y_train,
    kriging_mean,
    kriging_std,
    f"Sequential Kriging: local approach - Rank=0, RMSE={rmse_basic}",
)


We iterate sequentially and at each iteration, we add the point that currently has the highest Kriging error.

This added point is then included in our training set and GPR is fitted once again.

In [ ]:
n_iter = 6
X_train_local = X_train.copy()
y_train_local = y_train.copy()
kriging_std_local = kriging_std.copy()
gaussian_process_local = gaussian_process

for iter in range(n_iter):

    rng_max = kriging_std_local.argmax()
    new_x = X[rng_max]
    new_y = y[rng_max]
    X_train_local = np.append(X_train_local, [new_x], axis=0)
    y_train_local = np.append(y_train_local, new_y)

    gaussian_process.fit(X_train_local, y_train_local)
    kriging_mean_local, kriging_std_local = gaussian_process.predict(X, return_std=True)

    rmse_local = round(utils.rmse(y, kriging_mean_local, xlim, x_nbr), 2)

    plots.show_basic_kriging_improvement_1D(
        X,
        y,
        X_train_local,
        y_train_local,
        kriging_mean_local,
        kriging_std_local,
        new_x,
        new_y,
        f"Sequential Kriging: local approach - Rank={iter + 1}, RMSE={rmse_local}",
    )


Now let's see how far we've got by generating a few new simulations:

In [ ]:
print(f"Final RMSE after {n_iter} iterations: {rmse_local}")


### Global Approach

Choosing a point that will reduce the overall uncertainty most.

First, let's recall where we begin.

In [ ]:
plots.show_basic_kriging_plot_1D(
    X,
    y,
    X_train,
    y_train,
    kriging_mean,
    kriging_std,
    f"Sequential Kriging: global approach - Rank=0, RMSE={rmse_basic}",
)


In [ ]:
X_train_global = X_train.copy()
y_train_global = y_train.copy()
kriging_std_global = kriging_std.copy()
gaussian_process_global = gaussian_process


Let's say that we are choosing from the following 100 new potential points:

In [ ]:
X_try = np.linspace(start=0, stop=10, num=100).reshape(-1, 1)
y_try = gaussian_process.predict(X_try, return_std=False)


Compute integrated error when each try point is added.

Keep the same hyperparameters to fasten the model build.

In [ ]:
k1_new, k2_new = (
    gaussian_process.kernel_.k1.constant_value,
    gaussian_process.kernel_.k2.length_scale,
)
new_kernel = ConstantKernel(k1_new, constant_value_bounds="fixed") * RBF(
    length_scale=k2_new, length_scale_bounds="fixed"
)
gaussian_process_try = GaussianProcessRegressor(
    kernel=new_kernel, n_restarts_optimizer=9, optimizer=None
)


Loop over each hypothetical new points and compute resulting integrated error.

In [ ]:
integ_err = []
for i in range(len(X_try)):
    X_train_try = np.append(X_train_global, [X_try[i]], axis=0)
    y_train_try = np.append(y_train_global, y_try[i])
    gaussian_process_try.fit(X_train_try, y_train_try)
    _, kriging_std_try = gaussian_process_try.predict(X, return_std=True)
    integ_err.append(kriging_std_try.sum())


We iterate sequentially and at each iteration, we add the point that will most reduce the overall uncertainty.

This added point is then included in our training set and GPR is fitted once again.

Integrated error needs to be recalculated each time!

In [ ]:
n_iter = 6
for iter in range(n_iter):
    new_rng = np.array(integ_err).argmin()
    new_x = X_try[new_rng]
    new_y = utils.obj_function(new_x)
    y_try[new_rng] = new_y
    X_train_global = np.append(X_train_global, [new_x], axis=0)
    y_train_global = np.append(y_train_global, new_y)

    gaussian_process_global.fit(X_train_global, y_train_global)
    kriging_mean_global, kriging_std_global = gaussian_process_global.predict(
        X, return_std=True
    )

    rmse_global = round(utils.rmse(y, kriging_mean_global, xlim, x_nbr), 2)

    plots.show_basic_kriging_improvement_1D(
        X,
        y,
        X_train_global,
        y_train_global,
        kriging_mean_global,
        kriging_std_global,
        new_x,
        new_y,
        f"Sequential Kriging: global approach - Rank={iter + 1}, RMSE={rmse_global}",
    )

    k1_new, k2_new = (
        gaussian_process_global.kernel_.k1.constant_value,
        gaussian_process_global.kernel_.k2.length_scale,
    )
    new_kernel = ConstantKernel(k1_new, constant_value_bounds="fixed") * RBF(
        length_scale=k2_new, length_scale_bounds="fixed"
    )
    gaussian_process_try = GaussianProcessRegressor(
        kernel=new_kernel, n_restarts_optimizer=9, optimizer=None
    )

    integ_err = []
    for i in range(len(X_try)):
        X_train_try = np.append(X_train_global, [X_try[i]], axis=0)
        y_train_try = np.append(y_train_global, y_try[i])
        gaussian_process_try.fit(X_train_try, y_train_try)
        _, kriging_std_try = gaussian_process_try.predict(X, return_std=True)
        integ_err.append(kriging_std_try.sum())


Now let's see how far we've got by generating a few new simulations:

In [ ]:
print(f"Final RMSE after {n_iter} iterations: {rmse_global}")


### EGO

We aim to find the global maximum of the objective funciton.

In [ ]:
current_max_x = X_train[y_train.argmax()]
current_max = y_train.max()
print(f"current maximum is {round(current_max,2)}")


Now let's recall where we're at:

In [ ]:
plots.show_basic_kriging_plot_1D(
    X,
    y,
    X_train,
    y_train,
    kriging_mean,
    kriging_std,
    f"Sequential Kriging: Optimisation - Rank=0, current max: {round(current_max,2)}",
)


In [ ]:
X_train_optim = X_train.copy()
y_train_optim = y_train.copy()
kriging_mean_optim = kriging_mean.copy()
kriging_std_optim = kriging_std.copy()
gaussian_process_optim = gaussian_process


Now we will iteratively propose a potential new maximum, generate that point from the heavy simulator and repeat the process until the global maximum is reached within reasonable doubt.

The potential for improvement of each point is evaluated as the integral of its PDF above the current maximum.

In [ ]:
n_iter = 7

for iter in range(n_iter):

    ei_x = utils.expected_improvement(kriging_mean_optim, kriging_std_optim, current_max)
    rng_max_ei = ei_x.argmax()
    new_x = X[rng_max_ei]
    new_y = utils.obj_function(new_x)
    X_train_optim = np.append(X_train_optim, [new_x], axis=0)
    y_train_optim = np.append(y_train_optim, new_y)
    current_max_x = X_train_optim[y_train_optim.argmax()]
    current_max = y_train_optim.max()

    gaussian_process_optim.fit(X_train_optim, y_train_optim)
    kriging_mean_optim, kriging_std_optim = gaussian_process_optim.predict(X, return_std=True)

    plots.show_basic_kriging_improvement_1D(
        X,
        y,
        X_train_optim,
        y_train_optim,
        kriging_mean_optim,
        kriging_std_optim,
        new_x,
        new_y,
        f"Sequential Kriging: Optimisation - Rank={iter + 1}, current max: {round(current_max,2)}",
    )


There are no more points with a reasonable potential to exceed the current maximum.

The located maximum is:

In [ ]:
print(f"y = {round(current_max, 2)}")
print(f"x = {round(X[y==current_max][0][0], 2)}")


## Link to the practical part

https://colab.research.google.com/github/ml-kiwi-com/ml-prague/blob/main/02_practical_part.ipynb